In [13]:
import pandas as pd

# 1️⃣ Load the CSV file
df = pd.read_csv("C:\\MDA\\2025\\Term_1\\Principles of Analytics (DAMO 500-2)\\Final_Project\\Data\\98100482_Edition.csv")

# 2️⃣ Remove rows that contain "Total" in Gender or Education
df = df[~df['Gender (3)'].str.contains("Total", na=False)]
df = df[~df['Highest certificate, diploma or degree (16)'].str.contains("Total", na=False)]

# 3️⃣ Keep only the required columns (excluding DGUID, Totals, and Symbol)
columns_to_keep = [
    'REF_DATE', 'GEO', 'Gender (3)', 'Highest certificate, diploma or degree (16)',
    'Place of work status (7):Worked at home[2]',
    'Place of work status (7):Worked outside Canada[3]',
    'Place of work status (7):No fixed workplace address[4]',
    'Place of work status (7):Usual place of work[5]',
    'Place of work status (7):Usual place of work - Inside province/territory of residence[6]',
    'Place of work status (7):Usual place of work - Outside province/territory of residence[7]'
]

df = df[columns_to_keep]

# 4️⃣ Convert from wide to long format
df_long = pd.melt(
    df,
    id_vars=['REF_DATE','GEO','Gender (3)','Highest certificate, diploma or degree (16)'],
    var_name='Workplace_Status',
    value_name='Count'
)

# 4.1️⃣ Replace missing values in Count with 0
df_long['Count'] = df_long['Count'].fillna(0)

# 4.2️⃣ Filter only Toronto, excluding "Toronto (CMA)", and rename GEO → "Toronto"
df_long = df_long[
    df_long['GEO'].str.contains("Toronto", case=False, na=False) &
    ~df_long['GEO'].str.contains("Toronto \(CMA\)", case=False, na=False)
]
df_long['GEO'] = "Toronto"

# 5️⃣ Clean Workplace_Status column
df_long['Workplace_Status'] = df_long['Workplace_Status'].str.replace(
    'Place of work status \(7\):', '', regex=True
)
df_long['Workplace_Status'] = df_long['Workplace_Status'].str.replace(r'\[\d\]', '', regex=True)
df_long['Workplace_Status'] = df_long['Workplace_Status'].str.strip()

# 6️⃣ Clean Gender column (remove '+' at the end)
df_long['Gender (3)'] = df_long['Gender (3)'].str.replace(r'\+$', '', regex=True).str.strip()

# 7️⃣ Rename columns to English
df_long = df_long.rename(columns={
    'REF_DATE': 'Year',
    'GEO': 'City',
    'Gender (3)': 'Gender',
    'Highest certificate, diploma or degree (16)': 'Education_Level',
    'Workplace_Status': 'Workplace_Status',
    'Count': 'Count',
})

# 8️⃣ Create new column Workplace Location (grouping Remote / On Site)
workplace_map = {
    "No fixed workplace address": "Remote",
    "Worked at home": "Remote",
    "Worked outside Canada": "Remote",
    "Usual place of work": "On Site",
    "Usual place of work - Inside province/territory of residence": "On Site",
    "Usual place of work - Outside province/territory of residence": "On Site"
}
df_long["Workplace_Location"] = df_long["Workplace_Status"].map(workplace_map)

# 9️⃣ Check result
print(df_long.info())

# 🔟 Save the cleaned result to a new CSV
df_long.to_csv(
    "C:\\MDA\\2025\\Term_1\\Principles of Analytics (DAMO 500-2)\\Final_Project\\Data\\98100482_long_format_Toronto_cleaned.csv",
    index=False
)


<class 'pandas.core.frame.DataFrame'>
Index: 220860 entries, 123360 to 1104719
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Year                220860 non-null  int64  
 1   City                220860 non-null  object 
 2   Gender              220860 non-null  object 
 3   Education_Level     220860 non-null  object 
 4   Workplace_Status    220860 non-null  object 
 5   Count               220860 non-null  float64
 6   Workplace_Location  220860 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 13.5+ MB
None
